# Setup

In [12]:
# import your standard packages
%run ../../../global-tools/start.py

# make sure the figures plot inline rather than at the end
%matplotlib inline

Default libraries loaded.


In [13]:
# get data from tigress
# path = '/home/jennap/projects/LRGROUP/shared_data/ssh_AVISO/'
# slainfn = 'all_adt_sla_global_twosat_phy_l4_vDT2018_daily_1994_2020_03_01_dtrnd.nc'
ds = xr.open_dataset('sla_25km_processed.nc')

# Get data, selecting lat/lon slice
daily_sla = ds['daily_sla']
lat = daily_sla.lat.values
lon = daily_sla.lon.values
ds

<xarray.Dataset>
Dimensions:             (lat: 200, latitude: 200, lon: 340, longitude: 340, month: 12, month_time: 312, season: 4, season_time: 105, time: 9496)
Coordinates:
  * season_time         (season_time) datetime64[ns] 1993-12-01 ... 2019-12-01
  * longitude           (longitude) float64 35.12 35.38 35.62 ... 119.6 119.9
  * latitude            (latitude) float64 -19.88 -19.62 -19.38 ... 29.62 29.88
  * lon                 (lon) float64 35.12 35.38 35.62 ... 119.4 119.6 119.9
  * lat                 (lat) float64 -19.88 -19.62 -19.38 ... 29.38 29.62 29.88
  * time                (time) datetime64[ns] 1994-01-01 ... 2019-12-31
  * month_time          (month_time) datetime64[ns] 1994-01-01 ... 2019-12-01
  * month               (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * season              (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    daily_sla           (time, lat, lon) float64 ...
    mon_sla             (month_time, lat, lon) float64 ...
    seas_sla          

## AWG regridding

In [14]:
# get merged data from tigress
path = '../Merged/'
infn = 'Profiles_temp_psal_doxy_processed.nc'

ds_AWG = xr.open_dataset(path + infn)
ds_AWG

<xarray.Dataset>
Dimensions:  (pres: 201, time: 303903)
Coordinates:
    prof     (time) int32 ...
  * time     (time) datetime64[ns] 1958-06-01 1958-06-02 ... 2020-02-07
  * pres     (pres) int32 0 5 10 15 20 25 30 35 ... 970 975 980 985 990 995 1000
Data variables:
    temp     (time, pres) float32 ...
    psal     (time, pres) float32 ...
    doxy     (time, pres) float32 ...
    lat      (time) float32 ...
    lon      (time) float32 ...
    type     (time) int32 ...

In [15]:
# find unique time values of TCD
time_key = np.unique(ds_AWG.time)

# find all the days there are measurements of TCD
full_ind = np.full((daily_sla.shape[0]),False)

for tt in range(time_key.shape[0]):
    ind = daily_sla.time == time_key[tt]
    full_ind = full_ind + ind

# take only times where TCD measurements were made
sla_TCD = np.array(daily_sla[full_ind,:,:])
sla_time = np.array(daily_sla.time[full_ind])

xx,yy = np.meshgrid(lon,lat)
xx = xx.flatten()
yy = yy.flatten()

lons= np.array(ds_AWG.lon)
lats = np.array(ds_AWG.lat)
times = np.array(ds_AWG.time)

daily_sla_AWG = np.zeros(len(times))*np.nan
lon_AWG = np.zeros(len(times))*np.nan
lat_AWG = np.zeros(len(times))*np.nan
time_AWG = np.array(ds_AWG.time)

for tt in tqdm(range(sla_TCD.shape[0])):
    
    sla = np.array(sla_TCD[tt,:,:])
    indq = times == sla_time[tt]
    lonq = lons[indq]
    latq = lats[indq]

    # find the lons and lats of TCD vals at this time

    points = np.array( (xx,yy) ).T
    values = sla.flatten()


    daily_sla_AWG[indq] = griddata(points, values, (lonq,latq) ,method='linear' )
    lon_AWG[indq] = lonq
    lat_AWG[indq] = latq



100%|██████████| 7755/7755 [2:25:13<00:00,  1.10s/it]  


In [18]:
# convert to xarray dataset
ds=xr.Dataset(coords={'lon': lon,
                    'lat': lat,
                    'time': daily_sla.time})

# add variables to dataset

# AWG
ds["daily_sla_AWG"]=xr.DataArray(daily_sla_AWG,dims = ['time_AWG'],
                     coords =[time_AWG])
ds["lon_AWG"]=xr.DataArray(lon_AWG,dims = ['time_AWG'],
                     coords =[time_AWG])
ds["lat_AWG"]=xr.DataArray(lat_AWG,dims = ['time_AWG'],
                     coords =[time_AWG])

ds.to_netcdf('_processed.nc',mode='w',format = "NETCDF4")

ds

<xarray.Dataset>
Dimensions:        (lat: 200, lon: 340, time: 9496, time_AWG: 303903)
Coordinates:
  * lon            (lon) float64 35.12 35.38 35.62 35.88 ... 119.4 119.6 119.9
  * lat            (lat) float64 -19.88 -19.62 -19.38 ... 29.38 29.62 29.88
  * time           (time) datetime64[ns] 1994-01-01 1994-01-02 ... 2019-12-31
  * time_AWG       (time_AWG) datetime64[ns] 1958-06-01 ... 2020-02-07
Data variables:
    daily_sla_AWG  (time_AWG) float64 nan nan nan nan nan ... nan nan nan nan
    lon_AWG        (time_AWG) float64 nan nan nan nan nan ... nan nan nan nan
    lat_AWG        (time_AWG) float64 nan nan nan nan nan ... nan nan nan nan

In [30]:
daily_sla_AWG[~np.isnan(daily_sla_AWG)]

array([ 0.07522321,  0.05941689,  0.05987612, ..., -0.04433869,
       -0.06138302,  0.27052401])